# Collection des donnees(articles d'actualites)

In [35]:
import os
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from newspaper import Article

def fetch_articles(api_key, domains, save_path="corpus/article_actualite", metadata_file="all_documents.json"):
    """
    Récupère des articles à partir d'une API d'actualité, extrait leur contenu avec BeautifulSoup, 
    et enregistre les métadonnées dans un fichier JSON.

    Args:
        api_key (str): Clé API pour accéder à l'API des actualités.
        domains (list): Liste des catégories à récupérer (par ex. ['science', 'sports', 'technology']).
        save_path (str): Répertoire où sauvegarder les articles extraits.
        metadata_file (str): Fichier JSON pour sauvegarder les métadonnées des articles.
    """
    # Charger les métadonnées existantes ou initialiser une liste vide
    if os.path.exists(metadata_file):
        with open(metadata_file, "r", encoding="utf-8") as f:
            metadata = json.load(f)
    else:
        metadata = []

    article_id = len(metadata) + 1  # Commence l'ID à partir du dernier article

    base_url = "https://newsapi.org/v2/top-headlines"
    headers = {"Authorization": f"Bearer {api_key}"}

    for domain in domains:
        print(f"Récupération des articles pour la catégorie : {domain}")
        params = {"category": domain, "language": "en", "pageSize": 10}  # Limité à 10 articles par catégorie
        response = requests.get(base_url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Échec lors de la récupération des articles pour {domain} : {response.status_code}")
            continue

        articles = response.json().get("articles", [])
        domain_path = os.path.join(save_path, domain)
        os.makedirs(domain_path, exist_ok=True)

        for article in articles:
            url = article.get("url")
            title = article.get("title", "Sans titre")
            author = article.get("author", "Auteur inconnu")
            
            if not url:
                continue

            try:
                print(f"Traitement de l'URL : {url}")
                if url!="https://removed.com":
                    article_text = extract_text_from_url(url)
                    if article_text:
                        # Enregistrer l'article dans un fichier texte
                        file_path = save_article(domain_path, article_text, title)

                        # Ajouter les métadonnées au fichier JSON
                        metadata.append({
                        "id": article_id,
                        "title": title,
                        "author": author,
                        "type": "article d'actualité",
                        "categorie": domain,
                        "file_path": file_path,
                        "url": url
                        })
                        article_id += 1
            except Exception as e:
                print(f"Erreur lors du traitement de {url} : {e}")

    # Sauvegarder les métadonnées dans le fichier JSON
    with open(metadata_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    print(f"Les métadonnées ont été enregistrées dans {metadata_file}")

def extract_text_from_url(url):
    """
    Extrait le contenu textuel d'une URL donnée en utilisant Article.

    Args:
        url (str): URL de l'article.

    Returns:
        str:titre de l'article,str:auteur de l'article, str: Contenu textuel extrait.
    """
    article = Article(url)
    article.download()
    article.parse()
    #extraire le contenu 
    content = article.text
    return content


def save_article(path, text, title):
    """
    Sauvegarde le contenu de l'article dans un fichier texte.

    Args:
        path (str): Répertoire où sauvegarder l'article.
        text (str): Contenu de l'article.
        title (str): Titre de l'article (utilisé comme nom de fichier).

    Returns:
        str: Chemin complet du fichier sauvegardé.
    """
    safe_title = "".join(c if c.isalnum() else "_" for c in title)[:50]  # Nettoyer le titre pour en faire un nom de fichier valide
    file_path = os.path.join(path, f"{safe_title}.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)
    print(f"Article sauvegardé dans {file_path}")
    return file_path

# Application
if __name__ == "__main__":
    API_KEY = "35b99c0ae87b421390320689dee7fda3"  
    DOMAINS = ["business", "science", "sports", "technology"]  
    fetch_articles(API_KEY, DOMAINS)



Récupération des articles pour la catégorie : business
Traitement de l'URL : https://www.theverge.com/2024/12/23/24328046/walmart-spark-delivery-lawsuit-branch-instant-payment
Article sauvegardé dans corpus/article_actualite\business\Walmart_sued_over_illegally_opening_bank_accounts_.txt
Traitement de l'URL : https://www.marketwatch.com/story/stock-market-santa-claus-rally-might-get-a-late-start-this-year-18e58d0b
Erreur lors du traitement de https://www.marketwatch.com/story/stock-market-santa-claus-rally-might-get-a-late-start-this-year-18e58d0b : Article `download()` failed with 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/story/stock-market-santa-claus-rally-might-get-a-late-start-this-year-18e58d0b on URL https://www.marketwatch.com/story/stock-market-santa-claus-rally-might-get-a-late-start-this-year-18e58d0b
Traitement de l'URL : https://www.washingtonpost.com/technology/2024/12/23/arizona-data-centers-navajo-power-aps-srp/
Article sauvegardé dans corpus

# Collection de donnes (articles scientifiques)

In [39]:
import os
import requests
import json
from bs4 import BeautifulSoup


def save_article(keyword_path, abstract, title):
    """
    Sauvegarde le contenu d'un article scientifique dans un fichier texte.

    Args:
        keyword_path (str): Répertoire où sauvegarder l'article.
        abstract (str): Résumé de l'article.
        title (str): Titre de l'article.

    Returns:
        str: Chemin complet du fichier enregistré.
    """
    # Nettoyage du titre pour éviter les caractères non valides dans les noms de fichiers
    safe_title = re.sub(r'[^\w\s]', '', title).replace(" ", "_")
    file_name = f"{safe_title}.txt"
    file_path = os.path.join(keyword_path, file_name)

    # Écrire le résumé dans un fichier texte
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(abstract)

    return file_path


def fetch_scientific_articles(keywords, save_path="corpus/scientific_articles", metadata_file="all_documents.json"):
    """
    Récupère des articles scientifiques courts à partir de l'API ArXiv, extrait leur contenu, et enregistre les métadonnées.

    Args:
        keywords (list): Liste de mots-clés pour filtrer les articles (par ex. ['AI', 'biology', 'climate']).
        save_path (str): Répertoire où sauvegarder les articles scientifiques extraits.
        metadata_file (str): Fichier JSON pour sauvegarder les métadonnées des articles scientifiques.
    """
    # Charger ou initialiser les métadonnées
    if os.path.exists(metadata_file):
        with open(metadata_file, "r", encoding="utf-8") as f:
            metadata = json.load(f)
    else:
        metadata = []

    article_id = len(metadata) + 1  # ID unique pour chaque article

    base_url = "http://export.arxiv.org/api/query"

    for keyword in keywords:
        print(f"Récupération des articles scientifiques pour le mot-clé : {keyword}")
        params = {
            "search_query": f"all:{keyword}",
            "start": 0,
            "max_results": 5
        }
        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f"Échec lors de la récupération des articles pour {keyword} : {response.status_code}")
            continue

        # Parser la réponse XML avec BeautifulSoup
        soup = BeautifulSoup(response.content, "xml")
        entries = soup.find_all("entry")

        keyword_path = os.path.join(save_path, keyword)
        os.makedirs(keyword_path, exist_ok=True)

        for article in entries:
            title = article.find("title").get_text(strip=True)
            authors = ", ".join([author.find("name").get_text(strip=True) for author in article.find_all("author")])
            abstract = article.find("summary").get_text(strip=True)
            paper_url = article.find("id").get_text(strip=True)

            if not abstract or not paper_url:
                print(f"Article {title} ignoré : Résumé ou URL manquant.")
                continue

            try:
                print(f"Enregistrement de l'article : {title}")
                file_path = save_article(keyword_path, abstract, title)

                metadata.append({
                    "id": article_id,
                    "title": title,
                    "author": authors or "Auteur inconnu",
                    "type": "article scientifique",
                    "categorie": keyword,
                    "file_path": file_path,
                    "url": paper_url
                })
                article_id += 1
            except Exception as e:
                print(f"Erreur lors de l'enregistrement de {title} : {e}")

    # Sauvegarder les métadonnées dans un fichier JSON
    with open(metadata_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    print(f"Les métadonnées des articles scientifiques ont été enregistrées dans {metadata_file}")


# Application
if __name__ == "__main__":
    KEYWORDS = ["tech", "biology", "climate change", "quantum computing"]
    fetch_scientific_articles(KEYWORDS)


Récupération des articles scientifiques pour le mot-clé : tech
Enregistrement de l'article : The Contestation of Tech Ethics: A Sociotechnical Approach to Technology
  Ethics in Practice
Erreur lors de l'enregistrement de The Contestation of Tech Ethics: A Sociotechnical Approach to Technology
  Ethics in Practice : [Errno 22] Invalid argument: 'corpus/scientific_articles\\tech\\The_Contestation_of_Tech_Ethics_A_Sociotechnical_Approach_to_Technology\n__Ethics_in_Practice.txt'
Enregistrement de l'article : Exploring Multi-Dimensional Events Characterizing Tech Start-Up
  Emergence in the Nigerian Entrepreneurial Ecosystem
Erreur lors de l'enregistrement de Exploring Multi-Dimensional Events Characterizing Tech Start-Up
  Emergence in the Nigerian Entrepreneurial Ecosystem : [Errno 22] Invalid argument: 'corpus/scientific_articles\\tech\\Exploring_MultiDimensional_Events_Characterizing_Tech_StartUp\n__Emergence_in_the_Nigerian_Entrepreneurial_Ecosystem.txt'
Enregistrement de l'article : 


# Preprocessing des docs

In [51]:
import os
import json
import collections
from transformers import AutoTokenizer

# Charger le tokenizer de DistilBERT
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def read_documents(json_file_path):
    """
    Lit le fichier JSON, extrait les ids et les chemins des fichiers, lit le contenu de chaque fichier,
    et retourne un dictionnaire associant chaque id à son contenu.

    :param json_file_path: Chemin du fichier JSON contenant les métadonnées des documents (id, chemin du fichier).
    :return: Un dictionnaire {id: contenu du document}.
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            documents = json.load(json_file)
        
        doc_contents = {}
        for doc in documents:
            doc_id = doc.get("id")
            file_path = doc.get("file_path")
            # Vérification de l'existence du fichier et des informations nécessaires
            if doc_id and file_path and os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                doc_contents[doc_id] = content
            else:
                print(f"Fichier introuvable ou informations manquantes pour l'id {doc_id}: {file_path}")
        return doc_contents

    except Exception as e:
        print(f"Une erreur s'est produite lors de la lecture des documents : {e}")
        return {}

def preprocess_documents(documents_content):
    """
    Applique le prétraitement aux documents :
    - Tokenisation (utilisation de DistilBERT)
    - Construction du vocabulaire global à partir des tokens

    :param documents_content: Dictionnaire {id: contenu brut des documents}.
    :return: Dictionnaire avec le contenu prétraité et le vocabulaire global sous forme de Counter.
    """
    vocabulary = collections.Counter()
    preprocessed_content = {}

    for doc_id, content in documents_content.items():
        # Tokenisation du contenu du document avec gestion de la longueur des séquences
        tokens = tokenizer(content, truncation=True, padding=True, max_length=512, return_tensors="pt")
        
        # Enregistrer les tokens sous forme de dictionnaire avec input_ids et attention_mask
        preprocessed_content[doc_id] = {
            'input_ids': tokens['input_ids'].squeeze().tolist(),  # Convertir en liste simple
            'attention_mask': tokens['attention_mask'].squeeze().tolist()  # Convertir en liste simple
        }
        
        # Mise à jour du vocabulaire global
        vocabulary.update(tokens['input_ids'].squeeze().tolist())

    return preprocessed_content, vocabulary

def save_preprocessed_data(preprocessed_content, vocabulary, output_file="preprocessed_content.json"):
    """
    Sauvegarde le contenu prétraité et le vocabulaire dans un fichier JSON.

    :param preprocessed_content: Dictionnaire {id: contenu prétraité} (tokens).
    :param vocabulary: Vocabulaire global sous forme de `collections.Counter` (liste des ids).
    :param output_file: Chemin du fichier de sortie pour enregistrer les données.
    """
    data_to_save = {
        "preprocessed_content": preprocessed_content,
        "vocabulary": list(vocabulary.keys())  # Utiliser les clés du Counter comme vocabulaire
    }

    try:
        with open(output_file, 'w', encoding='utf-8') as json_file:
            json.dump(data_to_save, json_file, indent=4, ensure_ascii=False)
        print(f"Les contenus prétraités et le vocabulaire ont été enregistrés dans le fichier : {output_file}")
    except Exception as e:
        print(f"Une erreur s'est produite lors de la sauvegarde des données : {e}")

# Application principale
if __name__ == "__main__":
    json_file_path = "all_documents.json"
    documents_content = read_documents(json_file_path)

    if documents_content:
        preprocessed_content, vocabulary = preprocess_documents(documents_content)
        save_preprocessed_data(preprocessed_content, vocabulary)


Les contenus prétraités et le vocabulaire ont été enregistrés dans le fichier : preprocessed_content.json


# Representation des documents sous forme de vecteur

In [52]:
from transformers import AutoTokenizer, AutoModel
import torch
import json

# Charger le modèle et le tokenizer DistilBERT
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def load_preprocessed_content(input_file="preprocessed_content.json"):
    """
    Charge le contenu prétraité et le vocabulaire depuis un fichier JSON.

    :param input_file: Nom du fichier JSON contenant les données prétraitées et le vocabulaire.
    :return: Tuple (preprocessed_content, vocabulary) contenant les données prétraitées et le vocabulaire.
    """
    try:
        with open(input_file, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        preprocessed_content = data.get("preprocessed_content", {})
        vocabulary = data.get("vocabulary", [])
        return preprocessed_content, vocabulary
    except FileNotFoundError:
        print(f"Erreur : Le fichier {input_file} n'a pas été trouvé.")
        return {}, []
    except json.JSONDecodeError:
        print(f"Erreur : Le fichier {input_file} n'est pas un fichier JSON valide.")
        return {}, []
    except Exception as e:
        print(f"Une erreur s'est produite lors de la lecture du fichier : {e}")
        return {}, []

def create_distilbert_embeddings(doc_contents, output_file="distilbert_vectors.json"):
    """
    Utilise DistilBERT pour calculer des embeddings pour chaque document en moyennant les embeddings des tokens.

    :param doc_contents: Dictionnaire {id: contenu prétraité des documents (tokens)}.
    :param output_file: Nom du fichier JSON où les embeddings seront enregistrés.
    """
    try:
        if not doc_contents:
            print("Erreur : Aucun contenu de document fourni pour calculer les embeddings.")
            return

        document_vectors = {}

        for doc_id, tokens in doc_contents.items():
            # Vérification que tokens est bien un dictionnaire avec 'input_ids' et 'attention_mask'
            input_ids = tokens['input_ids']
            attention_mask = tokens['attention_mask']

            # Calcul des embeddings avec le modèle DistilBERT
            with torch.no_grad():
                # Assurez-vous que les tokens contiennent input_ids et attention_mask
                outputs = model(input_ids=torch.tensor([input_ids]),  # Ajouter une dimension batch
                                attention_mask=torch.tensor([attention_mask]))  # Ajouter une dimension batch
                embeddings = outputs.last_hidden_state.mean(dim=1)  # Moyenne des embeddings des tokens

            # Ajout des embeddings du document au dictionnaire
            document_vectors[doc_id] = embeddings.squeeze().tolist()

        # Enregistrer les vecteurs dans un fichier JSON
        with open(output_file, 'w', encoding='utf-8') as json_file:
            json.dump(document_vectors, json_file, indent=4)

        print(f"Les vecteurs DistilBERT des documents ont été enregistrés dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite lors du calcul des embeddings : {e}")

# Application principale
if __name__ == "__main__":
    # Charger les contenus prétraités
    preprocessed_content, vocabulary = load_preprocessed_content()

    # Vérifier si des contenus ont été chargés avec succès
    if preprocessed_content:
        # Créer les vecteurs avec DistilBERT
        create_distilbert_embeddings(preprocessed_content, output_file="distilbert_vectors.json")
    else:
        print("Impossible de générer les vecteurs car aucun contenu prétraité n'a été chargé.")


Les vecteurs DistilBERT des documents ont été enregistrés dans le fichier distilbert_vectors.json.


# Le calcule de similarites

In [54]:
import csv
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

def compute_similarity_matrix(vector_file="distilbert_vectors.json", output_file="document_similarity_matrix.csv"):
    """
    Calcule la similarité cosinus entre tous les documents et enregistre les résultats dans un fichier CSV sous forme de matrice.

    :param vector_file: Nom du fichier JSON contenant les vecteurs des documents.
    :param output_file: Nom du fichier CSV pour enregistrer la matrice des similarités.
    """
    try:
        # Charger les vecteurs depuis le fichier JSON
        with open(vector_file, 'r', encoding='utf-8') as json_file:
            document_vectors = json.load(json_file)
        
        # Extraire les IDs et les vecteurs
        doc_ids = list(document_vectors.keys())
        vectors = np.array([document_vectors[doc_id] for doc_id in doc_ids])
        
        # Calculer la matrice de similarité cosinus
        similarity_matrix = cosine_similarity(vectors)
        
        # Enregistrer la matrice dans un fichier CSV
        with open(output_file, 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            
            # Écrire l'en-tête (IDs des documents)
            writer.writerow([""] + doc_ids)
            
            # Écrire chaque ligne (ID + similarités)
            for i, doc_id in enumerate(doc_ids):
                writer.writerow([doc_id] + similarity_matrix[i].tolist())
        
        print(f"La matrice de similarité a été enregistrée dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")


def save_sorted_similarities_from_matrix(matrix_file="document_similarity_matrix.csv", output_file="sorted_document_similarities.csv"):
    """
    Enregistre dans un fichier CSV les IDs des documents les plus similaires pour chaque document,
    triés par similarité décroissante, à partir d'une matrice de similarité déjà calculée.

    :param matrix_file: Nom du fichier CSV contenant la matrice de similarité.
    :param output_file: Nom du fichier CSV pour enregistrer les similarités triées (seulement les IDs).
    """
    try:
        # Charger la matrice de similarité depuis le fichier CSV
        with open(matrix_file, 'r', encoding='utf-8') as csv_file:
            reader = csv.reader(csv_file)
            rows = list(reader)
        
        # Extraire les IDs des documents (en-tête)
        doc_ids = rows[0][1:]
        
        # Préparer les similarités triées (seulement les IDs)
        sorted_similarities = []
        for i, row in enumerate(rows[1:]):
            doc_id = row[0]
            similarities = [(doc_ids[j], float(row[j + 1])) for j in range(len(doc_ids)) if i != j]
            
            # Trier les similarités par ordre décroissant et récupérer uniquement les IDs
            sorted_doc_ids = [sim_doc_id for sim_doc_id, _ in sorted(similarities, key=lambda x: x[1], reverse=True)]
            sorted_similarities.append((doc_id, sorted_doc_ids))
        
        # Enregistrer les IDs triés dans un fichier CSV
        with open(output_file, 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            
            # Écrire l'en-tête (Document + Liste d'IDs des documents similaires)
            header = ["Document"] + [f"Similar Doc {i+1}" for i in range(len(doc_ids) - 1)]
            writer.writerow(header)
            
            # Écrire les similarités triées pour chaque document
            for doc_id, sorted_doc_ids in sorted_similarities:
                # Compléter avec des colonnes vides si moins de documents similaires
                row = [doc_id] + sorted_doc_ids + [''] * (len(doc_ids) - 1 - len(sorted_doc_ids))
                writer.writerow(row)
        
        print(f"Les similarités triées ont été enregistrées dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Application
if __name__ == "__main__":
    compute_similarity_matrix(vector_file="distilbert_vectors.json", output_file="document_similarity_matrix.csv")
    save_sorted_similarities_from_matrix(
        matrix_file="document_similarity_matrix.csv",
        output_file="sorted_document_similarities.csv"
    )


La matrice de similarité a été enregistrée dans le fichier document_similarity_matrix.csv.
Les similarités triées ont été enregistrées dans le fichier sorted_document_similarities.csv.
